In [258]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import folium
import random
import fontawesome as fa
from src.api import *
from src.clean import *
from src.mongodb import *

In [2]:
# {$and:[{},{}]}
# {$and:[{$or:[{},{},{},{}]},{}]}

In [3]:
db, coll = connectCollection('companies','companies_cleaned')

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [5]:
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies_cleaned')

In [6]:
companies = list(coll.find())

In [7]:
len(companies) # 9025

9025

In [8]:
categories = []
for i in range(len(companies)):
    categories.append(companies[i]['category_code'])
print(set(categories))

{'messaging', 'consulting', 'finance', 'hardware', 'manufacturing', 'cleantech', 'news', 'network_hosting', 'analytics', 'other', 'health', 'real_estate', 'security', 'transportation', 'music', 'advertising', 'design', 'games_video', 'semiconductor', 'fashion', 'education', None, 'software', 'biotech', 'local', 'search', 'automotive', 'sports', 'photo_video', 'mobile', 'web', 'social', 'nonprofit', 'nanotech', 'travel', 'hospitality', 'legal', 'enterprise', 'public_relations', 'medical', 'ecommerce'}


In [9]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
# The CEO is Vegan
# All people in the company have between 25 and 40 years, give them some place to go to party.
# Account managers need to travel a lot
# 30% of the company have at least 1 child.
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

In [10]:
# active_companies = list(coll.find({'deadpooled_year': np.nan}))

In [11]:
# len(active_companies) # 8406

In [12]:
# Nobody in the company likes to have companies with more than (10) years in a radius of 2 KM.
# {$and:[{deadpooled_year: NaN},{founded_year:{$lte:2009}}]}

# old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2009}}]}))

In [13]:
years= 13
old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2019-years}}]}))

In [14]:
len(old_companies)

4082

In [15]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.

# {$and:[{$or:[{category_code:'hardware'},{category_code:'nanotech'},{category_code:'mobile'},
# {category_code:'games_video'},{category_code:'cleantech'},{category_code:'software'},{category_code:'analytics'},
# {category_code:'web'},{category_code:'biotech'}]},{deadpooled_year: NaN},{founded_year:{$gte:2010}},
# {total_money_raised_USD:{$gte:1000000}}]}

# http://www.startup-book.com/2016/05/22/when-was-the-word-start-up-first-used/

successful_tech_startups = list(coll.find({'$and':[{'$or':[
        {'category_code':'semiconductor'},{'category_code':'network_hosting'},{'category_code':'consulting'},
        {'category_code':'design'},{'category_code':'hardware'},{'category_code':'nanotech'},
        {'category_code':'mobile'},{'category_code':'games_video'},{'category_code':'cleantech'},
        {'category_code':'software'},{'category_code':'analytics'},{'category_code':'web'},
        {'category_code':'biotech'}]},
    {'deadpooled_year': np.nan},{'founded_year':{'$gte':1999}},{'total_money_raised_USD':{'$gte':1000000}}]}))

# {'category_code':'ecommerce'},{'category_code':'photo_video'},{'category_code':'automotive'},{'category_code':'security'},


In [16]:
print(len(successful_tech_startups))

1119


In [17]:
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
# design_companies = list(coll.find({'category_code':'design'}))

In [18]:
# print(len(design_companies))

In [19]:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
# import geopy.distance
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)
# print geopy.distance.vincenty(coords_1, coords_2).km

In [20]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
possible_offices_criterion_1 = []
for e in successful_tech_startups:
    near_companies = getCompaniesNear(e['office_1_location']['coordinates'][0],
                                      e['office_1_location']['coordinates'][1],2000)
    # [near_companies[i] for i in range(len(near_companies)) if near_companies[i]['name'] == e['name']]
    # [near_companies[i] for i in range(len(near_companies)) if near_companies[i]['name'] != e['name']]
    for f in near_companies:
        if f['_id'] != e['name']:
            possible_offices_criterion_1.append(f['_id'])

In [21]:
print(len(list(set(possible_offices_criterion_1)))) # 5103 oficinas tienen cerca una startup

4798


In [22]:
possible_offices_c1 = list(set(possible_offices_criterion_1))

In [23]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.

not_possible_offices_criterion_2 = []
for e in old_companies:
    near_companies = getCompaniesNear(e['office_1_location']['coordinates'][0],
                                      e['office_1_location']['coordinates'][1],2000)
    for f in near_companies:
        if f['_id'] != e['name']:
            not_possible_offices_criterion_2.append(f['_id'])

In [24]:
print(len(list(set(not_possible_offices_criterion_2)))) # 8476/7358 oficinas tienen cerca una compañía antigua: 10/13 años

7358


In [25]:
not_possible_offices_c2 = list(set(not_possible_offices_criterion_2))

In [26]:
possible_offices_c1_c2 = []
for e in possible_offices_c1:
    if e not in not_possible_offices_c2:
        possible_offices_c1_c2.append(e)        
possible_offices_c1_c2_to_string = [str(e) for e in possible_offices_c1_c2]

In [27]:
print(len(possible_offices_c1_c2))
print(len(possible_offices_c1_c2_to_string))

85
85


In [30]:
df = pd.read_csv('./input/companies_df.csv', low_memory=False)
cols = ['_id', 'name', 'founded_year', 'category_code','deadpooled_year', 'total_money_raised_USD', 'num_offices',
 'office_1_longitude','office_1_latitude', 'office_1_location', 'office_1_city','office_1_state_code',
 'office_1_country_code']
df = df[cols]
df.dtypes

_id                        object
name                       object
founded_year              float64
category_code              object
deadpooled_year           float64
total_money_raised_USD    float64
num_offices                 int64
office_1_longitude        float64
office_1_latitude         float64
office_1_location          object
office_1_city              object
office_1_state_code        object
office_1_country_code      object
dtype: object

In [31]:
indexs = []
for i in range(len(df)):
    if df['_id'][i] in possible_offices_c1_c2_to_string:
        indexs.append(i)
print(len(indexs))
# print(indexs)

85


In [32]:
df_filtered = df.iloc[indexs]

In [33]:
df_filtered.head()

,_id,name,founded_year,category_code,deadpooled_year,total_money_raised_USD,num_offices,office_1_longitude,office_1_latitude,office_1_location,office_1_city,office_1_state_code,office_1_country_code
454,52cdef7c4bab8bd675297f9e,GENWI,2010.0,mobile,NaN,7100000.0,1,-111.903500,33.817100,"{'type': 'Point', 'coordinates': [-111.9035, 3...",Los Altos,CA,USA
478,52cdef7c4bab8bd675297fc4,ChessPark,2007.0,games_video,NaN,1000000.0,1,-83.913153,43.670840,"{'type': 'Point', 'coordinates': [-83.913153, ...",West Palm Beach,CA,USA
669,52cdef7c4bab8bd6752980be,Cyberhomes,NaN,web,NaN,0.0,1,-81.674745,30.318112,"{'type': 'Point', 'coordinates': [-81.674745, ...",Jacksonville,FL,USA
736,52cdef7c4bab8bd67529811d,RedLasso,NaN,games_video,NaN,8500000.0,1,-75.395990,40.099703,"{'type': 'Point', 'coordinates': [-75.39599, 4...",King of Prussia,PA,USA
755,52cdef7c4bab8bd675298139,divorce360,2007.0,web,NaN,2500000.0,1,-80.058401,26.820375,"{'type': 'Point', 'coordinates': [-80.058401, ...",North Palm Beach,FL,USA


In [52]:
possible_offices_c1_c2[0:10]

[ObjectId('52cdef7f4bab8bd67529c1ad'),
 ObjectId('52cdef7d4bab8bd67529960d'),
 ObjectId('52cdef7d4bab8bd67529a0d1'),
 ObjectId('52cdef7d4bab8bd675299c4f'),
 ObjectId('52cdef7c4bab8bd675298590'),
 ObjectId('52cdef7c4bab8bd6752986aa'),
 ObjectId('52cdef7e4bab8bd67529a9c4'),
 ObjectId('52cdef7f4bab8bd67529c062'),
 ObjectId('52cdef7d4bab8bd67529a00e'),
 ObjectId('52cdef7e4bab8bd67529a2c6')]

In [47]:
possible_offices_c1_c2_coords = []
for f in possible_offices_c1_c2:
    for e in companies:
        if e['_id'] == f:
            possible_offices_c1_c2_coords.append(e['office_1_location'])

In [48]:
print(len(possible_offices_c1_c2_coords))

85


In [53]:
possible_offices_c1_c2_coords[0:10]

[{'type': 'Point', 'coordinates': [-106.6634565, 52.143886]},
 {'type': 'Point', 'coordinates': [-87.596232, 41.78942]},
 {'type': 'Point', 'coordinates': [-90.082202, 29.973994]},
 {'type': 'Point', 'coordinates': [-81.657707, 30.310978]},
 {'type': 'Point', 'coordinates': [-79.842487, 40.517948]},
 {'type': 'Point', 'coordinates': [-118.422495, 34.153588]},
 {'type': 'Point', 'coordinates': [-111.81104, 40.718033]},
 {'type': 'Point', 'coordinates': [-75.413354, 40.100058]},
 {'type': 'Point', 'coordinates': [-86.0547016, 39.9052263]},
 {'type': 'Point', 'coordinates': [-87.5962319, 41.7894201]}]

In [54]:
df[df['_id'] == '52cdef7e4bab8bd67529a2c6']

,_id,name,founded_year,category_code,deadpooled_year,total_money_raised_USD,num_offices,office_1_longitude,office_1_latitude,office_1_location,office_1_city,office_1_state_code,office_1_country_code
5158,52cdef7e4bab8bd67529a2c6,CaptainU,2007.0,software,NaN,0.0,1,-87.596232,41.78942,"{'type': 'Point', 'coordinates': [-87.5962319,...",San Francisco,CA,USA


In [40]:
possible_offices_c1_c2_coords[0]['coordinates'][1] # latitud
possible_offices_c1_c2_coords[0]['coordinates'][0] # longitud

-111.9035

In [138]:
# for e in possible_offices_c1_c2_coords:
    # print(e['coordinates'][::-1])

In [55]:
# Starbucks a menos de 1 km:
starbucks_list = []
for i in range(len(possible_offices_c1_c2_coords)):
    data = foursquare_request_venues_authorized('search', possible_offices_c1_c2_coords[i]['coordinates'][1],
                                            possible_offices_c1_c2_coords[i]['coordinates'][0], 'starbucks',
                                            radius = 1000)
    starbucks_list.append([possible_offices_c1_c2[i],data])

In [ ]:
str(starbucks_list[0][0]) # ObjectId
starbucks_list[0][1]['response']['venues'][0]['location']['lat']
starbucks_list[0][1]['response']['venues'][0]['location']['lng']
# starbucks_list[0][1]['response']['venues'][0]['location']['labeledLatLngs']
starbucks_list[0][1]['response']['venues'][0]['location']['distance']
starbucks_list[0][1]['response']['venues'][0]['location']['formattedAddress']

In [109]:
cleaned_starbucks_list = []
for i in range(len(starbucks_list)):
    if len(starbucks_list[i][1]['response']['venues']) > 0:
        cleaned_starbucks_list.append([starbucks_list[i][0],
                                       starbucks_list[i][1]['response']['venues'][0]['location']['lat'],
                                       starbucks_list[i][1]['response']['venues'][0]['location']['lng'],
                                       starbucks_list[i][1]['response']['venues'][0]['location']['distance'],
                                       starbucks_list[i][1]['response']['venues'][0]['location']['formattedAddress']])

In [110]:
len(cleaned_starbucks_list)

47

In [111]:
possible_offices_c1_c2_c3 = [e[0] for e in cleaned_starbucks_list]
possible_offices_c1_c2_c3_to_string = [str(e) for e in possible_offices_c1_c2_c3]
# possible_offices_c1_c2_c3

In [112]:
indexs = []
for i in range(len(df)):
    if df['_id'][i] in possible_offices_c1_c2_c3_to_string:
        indexs.append(i)
print(len(indexs))
# print(indexs)

47


In [113]:
df_filtered = df.iloc[indexs]

In [123]:
len(df_filtered)

47

In [115]:
possible_offices_c1_c2_c3_coords = []
for f in possible_offices_c1_c2_c3:
    for e in companies:
        if e['_id'] == f:
            possible_offices_c1_c2_c3_coords.append(e['office_1_location'])

In [118]:
print(len(possible_offices_c1_c2_c3_coords))
possible_offices_c1_c2_c3_coords[0]

47


{'type': 'Point', 'coordinates': [-106.6634565, 52.143886]}

In [179]:
# Veganos a menos de 1 km:
vegan_list = []
for i in range(len(possible_offices_c1_c2_c3_coords)):
    data = foursquare_request_venues_authorized('search', possible_offices_c1_c2_c3_coords[i]['coordinates'][1],
                                            possible_offices_c1_c2_c3_coords[i]['coordinates'][0], 'vegan',
                                            radius = 2000)
    vegan_list.append([possible_offices_c1_c2_c3[i],data])

In [198]:
cleaned_vegan_list = []
for i in range(len(vegan_list)):
    if len(vegan_list[i][1]['response']['venues']) > 0:
        cleaned_vegan_list.append([vegan_list[i][0],
                                   vegan_list[i][1]['response']['venues'][0]['location']['lat'],
                                   vegan_list[i][1]['response']['venues'][0]['location']['lng'],
                                   vegan_list[i][1]['response']['venues'][0]['location']['distance'],
                                   vegan_list[i][1]['response']['venues'][0]['location']['formattedAddress'],
                                   vegan_list[i][1]['response']['venues'][0]['name']])

In [199]:
len(cleaned_vegan_list)

5

In [200]:
possible_offices_c1_c2_c3_c4 = [e[0] for e in cleaned_vegan_list]
possible_offices_c1_c2_c3_c4_to_string = [str(e) for e in possible_offices_c1_c2_c3_c4]

In [201]:
indexs = []
for i in range(len(df)):
    if df['_id'][i] in possible_offices_c1_c2_c3_c4_to_string:
        indexs.append(i)
print(len(indexs))
# print(indexs)

5


In [202]:
df_filtered = df.iloc[indexs]

In [203]:
len(df_filtered)

5

In [204]:
possible_offices_c1_c2_c3_c4_coords = []
for f in possible_offices_c1_c2_c3_c4:
    for e in companies:
        if e['_id'] == f:
            possible_offices_c1_c2_c3_c4_coords.append(e['office_1_location'])

In [205]:
print(len(possible_offices_c1_c2_c3_c4_coords))
possible_offices_c1_c2_c3_c4_coords[0]

5


{'type': 'Point', 'coordinates': [-106.6634565, 52.143886]}

In [266]:
# Clubes nocturnos:

# 4bf58dd8d48988d1ed931735 Aeropuerto
# 4bf58dd8d48988d1d3941735 Restaurante vegetariano/vegano
# 4bf58dd8d48988d11f941735 Local nocturno
# 4f4533804b9074f6e4fb0105 Escuela primaria
# 4bf58dd8d48988d13d941735 Colegio secundario
# Escuela intermedia 4f4533814b9074f6e4fb0106
# Jardín de infantes 4f4533814b9074f6e4fb0107
# Preescolar 52e81612bcbc57f1066b7a45
# Colegio privado 52e81612bcbc57f1066b7a46

party_list = VenuesListbtcategory(possible_offices_c1_c2_c3_c4_coords, possible_offices_c1_c2_c3_c4, '4bf58dd8d48988d11f941735', 1000)


In [272]:
party_list[0][1]['response']['venues'][0]

{'id': '4ccd1c05ee23a143cdec23a8',
 'name': 'Club Caribbean New Orleans Reggae Club',
 'contact': {},
 'location': {'address': '2441 Bayou Rd',
  'lat': 29.975514439982057,
  'lng': -90.07573797902866,
  'labeledLatLngs': [{'label': 'display',
    'lat': 29.975514439982057,
    'lng': -90.07573797902866}],
  'distance': 645,
  'postalCode': '70119',
  'cc': 'US',
  'city': 'New Orleans',
  'state': 'LA',
  'country': 'United States',
  'formattedAddress': ['2441 Bayou Rd',
   'New Orleans, LA 70119',
   'United States']},
 'categories': [{'id': '4bf58dd8d48988d11f941735',
   'name': 'Nightclub',
   'pluralName': 'Nightclubs',
   'shortName': 'Nightclub',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},


In [222]:
# Select one random row:
rowindex = random.choice(range(0,len(df_filtered)))
# df_filtered.iloc[rowindex][0] # id
# df_filtered.iloc[rowindex][7] # long
# df_filtered.iloc[rowindex][8] # lat
# df_filtered.iloc[rowindex][9] # location
# df_filtered.iloc[rowindex][10] # city
# df_filtered.iloc[rowindex][11] # state_code
# df_filtered.iloc[rowindex][12] # country_code

e = [df_filtered.iloc[rowindex][8], df_filtered.iloc[rowindex][7]]
print(rowindex)
print(e)

3
[49.8740096, 8.6605881]


In [223]:
df_filtered.iloc[rowindex]

_id                                                52cdef7e4bab8bd67529b333
name                                                          iPharro Media
founded_year                                                           2007
category_code                                                      software
deadpooled_year                                                         NaN
total_money_raised_USD                                             5.18e+06
num_offices                                                               2
office_1_longitude                                                  8.66059
office_1_latitude                                                    49.874
office_1_location         {'type': 'Point', 'coordinates': [8.6605881, 4...
office_1_city                                                     Darmstadt
office_1_state_code                                                     NaN
office_1_country_code                                                   DEU
Name: 7024, 

In [224]:
lat_long_starbucks = []
distance_starbucks = 0
for i in range(len(cleaned_starbucks_list)):
    if str(cleaned_starbucks_list[i][0]) == df_filtered.iloc[rowindex][0]:
        # print(cleaned_starbucks_list[i][1]) # lat
        # print(cleaned_starbucks_list[i][2]) # long
        # print(cleaned_starbucks_list[i][3]) # distance
        # print(cleaned_starbucks_list[i][4]) # adress
        lat_long_starbucks.append(cleaned_starbucks_list[i][1])
        lat_long_starbucks.append(cleaned_starbucks_list[i][2])
        distance_starbucks += cleaned_starbucks_list[i][3]
print(lat_long_starbucks)
print(distance_starbucks)

[49.87269833358546, 8.65098921152148]
703


In [225]:
lat_long_vegan = []
distance_vegan = 0
name_vegan = ''
for i in range(len(cleaned_vegan_list)):
    if str(cleaned_vegan_list[i][0]) == df_filtered.iloc[rowindex][0]:
        lat_long_vegan.append(cleaned_vegan_list[i][1])
        lat_long_vegan.append(cleaned_vegan_list[i][2])
        distance_vegan += cleaned_vegan_list[i][3]
        name_vegan += cleaned_vegan_list[i][5]
print(lat_long_vegan)
print(distance_vegan)
print(name_vegan)

[49.881272, 8.649491]
1134
Vrau Dötsch Veganer Fachhandel


In [226]:
print(f"The perfect location for your business is in {df_filtered.iloc[rowindex][10]}, {df_filtered.iloc[rowindex][12]}.")
print(f"You don't have companies with more than {years} years in a radius of 2 KM (blue circle).")
print(f"The office is near successful tech startups that have raised at least 1 Million dollars.")
print(f"Your employees will find a Starbucks just {distance_starbucks} meters from the office.")
print(f"The vegan restaurant '{name_vegan}' can be found just {distance_vegan} meters from the office.")

The perfect location for your business is in Darmstadt, DEU.
You don't have companies with more than 13 years in a radius of 2 KM (blue circle).
The office is near successful tech startups that have raised at least 1 Million dollars.
Your employees will find a Starbucks just 703 meters from the office.
The vegan restaurant 'Vrau Dötsch Veganer Fachhandel' can be found just 1134 meters from the office.


In [246]:
# {'gray', 'cadetblue', 'lightblue', 'darkblue', 'white', 'purple', 'green', 'lightred', 'pink', 'darkgreen',
# 'beige', 'darkred', 'red', 'orange', 'darkpurple', 'black', 'lightgreen', 'blue', 'lightgray'}

# map_city
tooltip = 'Click me!'
map_city = folium.Map(location = e, zoom_start=13)
folium.Circle(radius=2000,location=e,popup='Old companies free zone',color='#3186cc',
    fill=True,fill_color='#3186cc').add_to(map_city)
folium.Marker(e,radius=2,icon=folium.Icon(
    icon='briefcase', color='red'),popup='<b>Perfect location for your business</b>',
    tooltip=tooltip).add_to(map_city)
folium.Marker(lat_long_starbucks,radius=2,icon=folium.Icon(
    icon='coffee', prefix='fa',color='orange'),popup='Starbucks',
    tooltip=tooltip).add_to(map_city)
folium.Marker(lat_long_vegan,radius=2,icon=folium.Icon(
    icon='cutlery',color='green'),popup=f"Vegan restaurant: '{name_vegan}'",
    tooltip=tooltip).add_to(map_city)
map_city
# map_city.save('index.html')